In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
# Only read this in once or it will return a NULL. Read in data from streaming_body_1.
readrawdata = streaming_body_1.read()

In [3]:
# StringIO needed to convert bytes file to pandas dataframe.
from io import StringIO

# Convert bytes file to pandas dataframe.
s=str(readrawdata,'utf-8')
data = StringIO(s) 
#First two hundred value in dataset.
df=pd.read_csv(data, sep="\t", header=None)
# As discussed in "Data exploration", this is a pretty ideal chunk of training data, 240,000 rows long, to use to translate phrases from children's books.
df = df[120000:360000]
print(df.shape)



(240000, 3)


In [4]:
# To save memory, delete unecessary variables from memory. 
del s, data, readrawdata, endpoint_c9a37f3a569f4269ad9b640bc13792f3, client_c9a37f3a569f4269ad9b640bc13792f3, streaming_body_1

In [5]:
# Delete attribution column
df = df.drop(df.columns[[2]], axis=1)

In [6]:
# Add column names
df.columns = ['English', 'Russian']

In [7]:
# Install Tensorflow, the framework I'll be using for this project. 
!pip install tensorflow==2.6.0


/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 458.3 MB 17 kB/s s eta 0:00:01
     |████████████████████████████████| 4.0 MB 49.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 43.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 40.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 41.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 6.6 MB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 2.5 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 50.3 MB/s eta 0:

In [8]:
# Import necessary tensorflow packages and numpy libraries. 
import tensorflow as tf
import numpy as np
from numpy import array
from numpy import argmax
from numpy import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [9]:
import string
# Name columns for clarity
# Clean data by lowercasing and removing punctuation

df = df.apply(lambda x: x.astype(str).str.lower())
df = df.apply(lambda x: x.astype(str).str.replace(r'[^\w\s]+', ''))

print(df.head())

                        English                      Russian
120000  no one owes me anything   никто мне ничего не должен
120001  no one ran ahead of him  впереди него никто не бежал
120002  no one slept that night      в ту ночь никто не спал
120003  no one told me anything   никто ничего мне не сказал
120004  no one told me anything   никто мне ничего не сказал


In [10]:
# Functions. The grand purpose is to convert data to categorical, and output data to one-hot encoding. 
from tensorflow.keras.utils import to_categorical

# Create a tokenizer, which assigns each unique word to a unique integer. This is necessary for categorical encoding. 
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

''' Finds the longest sentence in the translation set. This is necessary for creating tokenizer arrays, because each tokenized row
will have length max_length, and will be padded with zeros.'''
def max_length(lines):
	return max(len(line.split()) for line in lines)

'''Categorically encode using max length. For example, the string "John went to the park." will be encoded as [123 3 5 2 435 0 0 0...]. 
The length will be max_length, and each integer in the list corresponds to a word in the sentence (as encoded by the tokenizer).'''
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

'''One-hot encoding. This will create a sparse matrix for every row in the output set, of length max_length. Each list in the matrix will have length 
vocab_size, and a 1 will be at the index corresponding to the word's encoded token.'''
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [11]:
# Create the tokenizer, calculate vocab case, and find max length of an English sentence in the set. All for future encoding.
eng_tokenizer = create_tokenizer(df["English"])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(df['English'])

In [12]:
# Create the tokenizer, calculate vocab case, and find max length of an Russian sentence in the set. All for future encoding.
rus_tokenizer = create_tokenizer(df['Russian'])
rus_vocab_size = len(rus_tokenizer.word_index) + 1
rus_length = max_length(df['Russian'])

In [13]:
# Uses the tokenizer to output a word with input of its index.
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

(240000,)
14793


In [14]:
# First, use train_test_split to establish target and source datasets for training and testing. 
from sklearn.model_selection import train_test_split

# Training set will have 192k values. Test set will have 48k.
x_train, x_test, y_train, y_test =  train_test_split(df['Russian'], df['English'], test_size=0.2)

''' Encode training datasets. I'm not one-hot-encoding the output because of the huge amount of memory required. I will be one-hot encoding the output
in chunks as I feed it into the model using a generator. '''
trainX = encode_sequences(rus_tokenizer, rus_length, x_train)
trainY = encode_sequences(eng_tokenizer, eng_length, y_train)
# 192,000 training cases. 58,000 test cases.

In [39]:
# prepare validation data
testX = encode_sequences(rus_tokenizer, rus_length, x_test[0:10000])
testY = encode_sequences(eng_tokenizer, eng_length, y_test[0:10000])
testY = encode_output(testY, eng_vocab_size)


In [22]:
# Import all of the Tensorflow packages I'll need for my model.
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [23]:
# Define my model. 
'''First, I use an Embedding layer, which creates a vector space in which words with similar meanings are mapped more closely to each other. 
A long short-term memory layer is a type of recurrent neural network that has feedback as well as feed forward connections. Long story short, 
the difference between a bidirectional LSTM and a conventional LSTM (unidirectional by default), is that a bidirectional LSTM runs inputs through
in two orders, enabling it to preserve both past and future data. The RepeatVector layer essentially adds another dimension to your model of dimension
batch_size. The TimeDistributed layer applies a layer for a specified slice of unit time. It wraps up the model by forming a vector of probabilities. 
The model is a sequential model with two Dropouts to prevent overfitting. I initially tried an LSTM, but research indicated that Bidirectional Long
Short-Term Models are usually preferable to LSTMs in Natural Language Processing contexts. 
'''
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(Bidirectional(LSTM(n_units)))
	model.add(Dropout(0.2))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

In [24]:
# I found that an n-gram value of 512 not only slowed down the fitting, but increased overfitting. 
# adam tends to be the preferred optimizer for NLP processes. 
model = define_model(rus_vocab_size, eng_vocab_size, rus_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [25]:
# Below is a nice little summary of the model.
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 256)           11665152  
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               1050624   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 17, 512)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 17, 256)           787456    
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 256)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 17, 14793)         3

In [26]:
# Install the two things that will allow me to plot the model. 
!pip install pydot
!pip install graphviz

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [27]:
# Plot the model.
import pydot
import graphviz
plot_model(model, to_file='model.png', show_shapes=True)


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [37]:
'''Evaluate the model using BLEU (bilingual evaluation understudy). In order to view the model outputs, we need a function for model predictions
to compare it to the test data. '''
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

def evaluate_model(model, tokenizer, sources):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = y_test.iloc[i], x_test.iloc[i]
		if i % 1000 == 0:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [30]:
# data generator, intended to be used in a call to model.fit_generator().
# The data generator one-hot encodes the output data, then feeds it into the model.
# The output data will be in tensor form. If I didn't use a generator, my data would be of size 192,000 x ~11,000 x 18 = ~38 billion. 
# You need a lot more memory than I have at my disposal to train a model with a sparse tensor of size 38 billion. 

def data_generator(source, target):
    # loop for ever over images
    while 1:
        for x in range(0, 3000):
            target_seg = encode_output(target[x*64:(x+1)*64], eng_vocab_size)
            source_seg = source[x*64:(x+1)*64]
            yield source_seg, target_seg

In [31]:
# Define the generator variable for use in model.fit
generator = data_generator(trainX, trainY)

In [34]:
# Define name of model to be stored in Object Storage. A checkpoint will be called after every epoch to calculate the loss using test data. 
# The model with the lowest calculated loss on the test data will be saved to Object storage. 
# The validation data is of size 2,000, because with my 16 GB of RAM, this seems to be roughly the maximum number that doesn't result in kernel restarts. 
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(generator, epochs=20, steps_per_epoch=3000, verbose=2, batch_size=64, callbacks=[checkpoint], validation_data=(testX, testY))


Epoch 1/20
3000/3000 - 1053s - loss: 2.5477 - val_loss: 2.1381

Epoch 00001: val_loss improved from inf to 2.13810, saving model to model.h5
Epoch 2/20
3000/3000 - 1033s - loss: 1.9820 - val_loss: 1.6826

Epoch 00002: val_loss improved from 2.13810 to 1.68255, saving model to model.h5
Epoch 3/20
3000/3000 - 1040s - loss: 1.5597 - val_loss: 1.3488

Epoch 00003: val_loss improved from 1.68255 to 1.34883, saving model to model.h5
Epoch 4/20
3000/3000 - 1044s - loss: 1.2800 - val_loss: 1.1721

Epoch 00004: val_loss improved from 1.34883 to 1.17215, saving model to model.h5
Epoch 5/20
3000/3000 - 1040s - loss: 1.1029 - val_loss: 1.0753

Epoch 00005: val_loss improved from 1.17215 to 1.07532, saving model to model.h5
Epoch 6/20
3000/3000 - 1033s - loss: 0.9819 - val_loss: 1.0192

Epoch 00006: val_loss improved from 1.07532 to 1.01916, saving model to model.h5
Epoch 7/20
3000/3000 - 1034s - loss: 0.8937 - val_loss: 0.9862

Epoch 00007: val_loss improved from 1.01916 to 0.98624, saving model t

KeyboardInterrupt: 

In [35]:
# Load that model from Object Storage. 
model = load_model('model.h5')


In [40]:
# Evaluate the model using BLEU. First it spits out some sample predictions. As you can see, the model does pretty well. 
# The upper bound on the BLEU score is > .6, which is considered to be "better than most translators."
evaluate_model(model, eng_tokenizer, testX)

src=[в ветровом стекле было пулевое отверстие], target=[there was a bullet hole in the windshield], predicted=[a was was a in in the window]
src=[деревня находится за теми деревьям], target=[the village is beyond those trees], predicted=[the is is the these trees are]
src=[я знаю что у тебя есть домашняя работа которую следует выполнить], target=[i know you have homework to do], predicted=[i know you have have homework do do]
src=[в прошлые выходные том был с семьёй], target=[tom was with his family last weekend], predicted=[tom was with last last last]
src=[мне надо чтобы ты поговорил с томом], target=[i need you to talk to tom], predicted=[i need you to talk to tom]
src=[я попала под ливень], target=[i was caught in a shower], predicted=[i was caught in a rainstorm]
src=[мы ходили вместе в библиотеку], target=[we went to the library together], predicted=[we went to the library library]
src=[я знал что нам надо было уйти раньше], target=[i knew we shouldve left earlier], predicted=[i 

In [49]:
evaluate_model(model, eng_tokenizer, trainX[0:10000])

src=[она пробудет в ньюйорке две недели], target=[she will be in new york for two weeks], predicted=[she will be in new york for a weeks]
src=[том закончил работу за три часа], target=[tom finished the job in three hours], predicted=[tom finished the work for three]
src=[том знает что мэри не говорит пофранцузски], target=[tom knows that mary cant speak french], predicted=[tom knows mary mary cant speak french]
src=[я лучше не буду смотреть этот фильм], target=[id rather not see that movie], predicted=[id rather be see that movie movie]
src=[он стал великим музыкантом], target=[he became a great musician], predicted=[he became a great musician]
src=[куда вы ездили в понедельник], target=[where did you go on monday], predicted=[where did you go on monday]
src=[том любит играть в баскетбол], target=[tom likes to play basketball], predicted=[tom likes playing play basketball]
src=[он казалось получает удовольствие от своей жизни и своей работы], target=[he seemed to enjoy his life and his

In [69]:
# The code was removed by Watson Studio for sharing.

,Phrases
0,Одеяло Убежало
1,Улетела простыня
2,И подушка
3,Как лягушка
4,Ускакала от меня


In [70]:
# To lower.
np_df['Phrases'] = np_df['Phrases'].str.lower()
# Get rid of numbers.
np_df['Phrases'] = np_df['Phrases'].str.replace('\d+', '')
# Get rid of non-Cyrillic characters. 
np_df['Phrases'] = np_df['Phrases'].str.replace(r'[^\w\s]+', '')
# Get rid of leading and trailing whitespace. 
np_df['Phrases'] = np_df['Phrases'].str.strip()
# Get rid of duplicate entries. 
np_df = np_df.drop_duplicates(subset=None, keep='first', inplace=False)
# Reset index. 
np_df = np_df.reset_index()
np_df = np_df.drop(columns = ['index'])
# This dataset has 799 entries, the other one has 1049. Let's cut them both to 750, so they're both the same size.
np_df = np_df.head(750)
np_df.tail()

,Phrases
745,и снова медведь
746,спасите моржа
747,вчера проглотил он морского ежа
748,и такая дребедень целый день
749,диньдилень


In [67]:
#Load propaganda dataset. 
body = client_c29a7a805d8d4196b70668aaf704d0e0.get_object(Bucket='default-donotdelete-pr-f13vke2upogt8x',Key='propaganda.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

p_df = pd.read_csv(body)
p_df.head()


,Phrases
0,Это первая советская фабрика чая
1,"Она находится в совхозе „Чаква"""
2,Чаква
3,станция закавказской железной дороги
4,Молодые листья чая


In [85]:
#Refer to above.
p_df['Phrases'] = p_df['Phrases'].str.lower()
p_df['Phrases'] = p_df['Phrases'].str.replace('\d+', '')
p_df['Phrases'] = p_df['Phrases'].str.replace(r'[^\w\s]+', '')
p_df['Phrases'] = p_df['Phrases'].str.strip()
p_df = p_df.drop_duplicates(subset=None, keep='first', inplace=False)
p_df = p_df.reset_index()
p_df = p_df.drop(columns = ['index'])
p_df = p_df.head(750)

In [91]:
source = encode_sequences(rus_tokenizer, rus_length, p_df.iloc[511])
#source = source.reshape((1, source.shape[0]))
translation = predict_sequence(model, eng_tokenizer, source)
print(translation)

bothers oral oral steep steep steep steep steep steep steep flush flush flush flush flush flush flush


In [89]:
print(p_df.iloc[510])


Phrases    не выпускайте винтовок из своих рук
Name: 510, dtype: object


In [71]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/wsuser/nltk_data...


True

In [72]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA


In [73]:
# testing
sia = SIA()
results = []
headlines = ["празднование столетия мю лермонтова отложено	", "Soldier surprises wife.", "Gas prices on the rise.", "Jews murdered in the streets.", "Osama Bin Laden is dead."]
for line in headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

print(results)


[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'празднование столетия мю лермонтова отложено\t'}, {'neg': 0.0, 'neu': 0.513, 'pos': 0.487, 'compound': 0.2263, 'headline': 'Soldier surprises wife.'}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'headline': 'Gas prices on the rise.'}, {'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'compound': -0.6597, 'headline': 'Jews murdered in the streets.'}, {'neg': 0.518, 'neu': 0.482, 'pos': 0.0, 'compound': -0.6486, 'headline': 'Osama Bin Laden is dead.'}]


In [137]:
print(news_df["title"][0])
#source = trainX[0].reshape((1, trainX[0].shape[0]))
#print(source)
#print(news_df["title"].iloc[0])

#print(news_df["title"][0])
#source = encode_sequences(eng_tokenizer, eng_length, news_df["title"][0])
#X = tokenizer.texts_to_sequences(lines)
# pad sequences with 0 values
#X = pad_sequences(X, maxlen=length, padding='post')

 русские войска вступили в пределы венгрии  
